<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0614/0614_02_4_RNN_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1(%EC%8B%A4%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 31.2 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Bidirectional, Dropout, GRU

In [ ]:
# 영화 리뷰 데이터
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

#### 데이터 확인 (리뷰)

In [ ]:
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


### 데이터 전처리

In [ ]:
from konlpy.tag import Okt

def text_preprocess(doc):
    from konlpy.tag import Okt
    import re

    tokenizer = Okt()

    # 텍스트 클리닝
    clean_doc = re.sub('[^가-힣ㄱ-ㅣa-zA-Z|0-9.%]', ' ', string = doc)

    # 품사 붙이기
    token_tagged = tokenizer.pos(clean_doc)

    # 명사추출
    nouns = [s for s, t in token_tagged if t in ['Noun'] and len(s) > 1]

    # 한 문장으로 합치기
    token_nouns = ' '.join(nouns)

    # TfidfVectorizer는 토큰화 하면 안되서 join으로 합치기
    if token_nouns :
        return ''.join(token_nouns)
    else :
        return np.nan

In [ ]:
from tqdm import tqdm
tqdm.pandas()

train['train'] = train['document'].progress_apply(lambda x : text_preprocess(x))
train['train']

100%|██████████| 149995/149995 [11:08<00:00, 224.36it/s]


0                      더빙 진짜 목소리
1             포스터 보고 초딩 영화 오버 연기
2                 무재 밓었 다그 래서 추천
3            교도소 이야기 구먼 재미 평점 조정
4         몬페 연기 영화 스파이더맨 커스틴 던스트
                   ...          
149995                  인간 문제 죄인
149996                        평점
149997            한국인 먹거리 필리핀 혼혈
149998          청춘 영화 최고봉 방황 자화상
149999         한국 영화 최초 수간 내용 영화
Name: train, Length: 149995, dtype: object

### 토크나이저

In [302]:
# 토크나이저 생성
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer_review = Tokenizer(split=' ', char_level=False)

In [325]:
x_data = train['train'].dropna()

In [326]:
x_data

0                      더빙 진짜 목소리
1             포스터 보고 초딩 영화 오버 연기
2                 무재 밓었 다그 래서 추천
3            교도소 이야기 구먼 재미 평점 조정
4         몬페 연기 영화 스파이더맨 커스틴 던스트
                   ...          
149995                  인간 문제 죄인
149996                        평점
149997            한국인 먹거리 필리핀 혼혈
149998          청춘 영화 최고봉 방황 자화상
149999         한국 영화 최초 수간 내용 영화
Name: train, Length: 139594, dtype: object

In [303]:
# 단어 인덱스 구축

tokenizer_review.fit_on_texts(x_data)

In [304]:
print(tokenizer_review.word_index)

{'영화': 1, '정말': 2, '진짜': 3, '연기': 4, '평점': 5, '최고': 6, '스토리': 7, '생각': 8, '드라마': 9, '감동': 10, '사람': 11, '보고': 12, '배우': 13, '감독': 14, '재미': 15, '내용': 16, '그냥': 17, '쓰레기': 18, '시간': 19, '작품': 20, '사랑': 21, '하나': 22, '다시': 23, '마지막': 24, '이건': 25, '완전': 26, '정도': 27, '처음': 28, '장면': 29, '액션': 30, '주인공': 31, '최악': 32, '지금': 33, '이야기': 34, '별로': 35, '느낌': 36, '연출': 37, '명작': 38, '역시': 39, '이해': 40, '이영화': 41, '여자': 42, '때문': 43, '보기': 44, '기억': 45, '결말': 46, '마음': 47, '인생': 48, '소재': 49, '수준': 50, '현실': 51, '한번': 52, '가장': 53, '반전': 54, '매력': 55, '전개': 56, '남자': 57, '한국': 58, '가슴': 59, '음악': 60, '아이': 61, '원작': 62, '인간': 63, '무슨': 64, '우리': 65, '추천': 66, '눈물': 67, '자체': 68, '캐릭터': 69, '코미디': 70, '대박': 71, '뭔가': 72, '대한': 73, '전혀': 74, '이상': 75, '연기력': 76, '여운': 77, '기대': 78, '모두': 79, '개봉': 80, '공감': 81, '일본': 82, '아주': 83, '중간': 84, '영상': 85, '시리즈': 86, '부분': 87, '모습': 88, '계속': 89, '제목': 90, '가족': 91, '내내': 92, '보지': 93, '진심': 94, '실망': 95, '다른': 96, '몰입': 97, '표현': 98, '기분': 99, '작가': 1

In [ ]:
vocab_size = len(tokenizer_review.word_index) +1
vocab_size

37126

### 훈련데이터/ test 데이터 생성

In [ ]:
# 훈련 데이터 만들기 1. 뒤에 단어 추측할 수 있도록 문장 분할한 리스트 생성

sequences_list = []

for doc in x_data:
    line = tokenizer_review.texts_to_sequences([doc])[0] # 단어 > 정수로 변환

    # line = doc.split(' ') # 한글로 보기 편하려면 이 코드 사용> 나중에 정수 변환 필요
    for i in range(1, len(line)):
        sequence = line[:i+1]
        sequences_list.append(sequence)

sequences_list

In [ ]:
print(sequences_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [282]:
# 훈련 데이터 만들기 2. 문장 길이가 균일하지 않아서 padding

padded_1 = pad_sequences(sequences_list, padding = 'pre')
padded_1

array([[   0,    0,    0, ...,    0, 4886,  618],
       [   0,    0,    0, ..., 4886,  618, 3066],
       [   0,    0,    0, ...,  618, 3066, 3048],
       ...,
       [   0,    0,    0, ...,   62, 3066,  506],
       [   0,    0,    0, ..., 3066,  506,  323],
       [   0,    0,    0, ...,  506,  323, 3066]], dtype=int32)

In [283]:
# 훈련 데이터 만들기 3. x 데이터와 y 데이터 분리

X_train_1 = padded_1[:, :-1]
y_train_1 = padded_1[:, -1]

In [284]:
print (X_train_1[0], '\n', y_train_1[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0 4886] 
 618


In [285]:
X_train_1.shape, y_train_1.shape

((248232, 64), (248232,))

In [ ]:
# test 데이터 만들기

test_tokenizer = Tokenizer(split=' ', char_level=False)
test_tokenizer.fit_on_texts(test['document'])


In [ ]:
x_test = test['document']

In [ ]:
test_tokenizer.word_index

In [ ]:
x_test = list(x_test)

x_test

In [ ]:
test_list = []

for doc in str(x_test).split(' '):
    line = tokenizer_review.texts_to_sequences([doc])[0] # 단어 > 정수로 변환

    # line = doc.split(' ') # 한글로 보기 편하려면 이 코드 사용> 나중에 정수 변환 필요
    for i in range(1, len(line)):
        sequence = line[:i+1]
        test_list.append(sequence)

test_list

In [ ]:
test_padded = pad_sequences(test_list, padding = 'pre')
test_padded

In [ ]:
X_test = padded_1[:, :-1]
y_test = padded_1[:, -1]

In [ ]:
print (X_test[0], '\n', y_test[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0 163] 
 3


In [ ]:
X_test.shape, y_test.shape

((555143, 65), (555143,))

In [ ]:
# def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
#     init_word = current_word
#     sentence = ''

#     # n번 반복
#     for _ in range(n):
#         # 현재 단어에 대한 정수 인코딩과 패딩
#         encoded = tokenizer.texts_to_sequences([current_word])[0]
#         encoded = pad_sequences([encoded], maxlen=4, padding='pre')

#         # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
#         result = model.predict(encoded, verbose=0)
#         result = np.argmax(result, axis=1)

#         for word, index in tokenizer.word_index.items():
#             # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
#             if index == result:
#                 break

#         # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
#         current_word = current_word + ' '  + word

#         # 예측 단어를 문장에 저장
#         sentence = sentence + ' ' + word

#     sentence = init_word + sentence
#     return sentence

### 모델생성

In [ ]:
# modeling_Sequential

# model = Sequential([
#     tf.keras.layers.Embedding(vocab_size, 15),
#     tf.keras.layers.GRU(50, return_sequences = False),
#     tf.keras.layers.Dense(vocab_size, activation = 'sofrmax')
# ])


In [290]:
# modeling

input = tf.keras.Input(shape = (64,))

a = tf.keras.layers.Embedding(vocab_size, 15)(input)
a = tf.keras.layers.GRU(30,return_sequences = False)(a)

output = tf.keras.layers.Dense(vocab_size, activation = 'softmax')(a)

model = tf.keras.Model(inputs = input, outputs = output)

In [291]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer ='adam',
              metrics = ['acc'])

In [322]:
model.fit(X_train_1[:100], y_train_1[:100], batch_size = 64, epochs = 100, validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 2s 1s/step - loss: 3.6570 - acc: 0.1375 - val_loss: 11.3849 - val_acc: 0.0500
Epoch 2/100
2/2 [==============================] - 0s 147ms/step - loss: 3.6577 - acc: 0.1375 - val_loss: 11.3863 - val_acc: 0.0500
Epoch 3/100
2/2 [==============================] - 0s 114ms/step - loss: 3.6569 - acc: 0.1375 - val_loss: 11.3866 - val_acc: 0.0500
Epoch 4/100
2/2 [==============================] - 0s 120ms/step - loss: 3.6570 - acc: 0.1375 - val_loss: 11.3866 - val_acc: 0.0500
Epoch 5/100
2/2 [==============================] - 0s 173ms/step - loss: 3.6571 - acc: 0.1375 - val_loss: 11.3876 - val_acc: 0.0500
Epoch 6/100
2/2 [==============================] - 0s 117ms/step - loss: 3.6569 - acc: 0.1375 - val_loss: 11.3891 - val_acc: 0.0500
Epoch 7/100
2/2 [==============================] - 0s 121ms/step - loss: 3.6562 - acc: 0.1375 - val_loss: 11.3902 - val_acc: 0.0500
Epoch 8/100
2/2 [==============================] - 0s 163ms/step - loss: 3.6562

### 테스트

In [318]:
X_train_1[21]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0, 1771, 3066], dtype=int32)

In [324]:
y_pred = model.predict(X_train_1[21:22])

1/1 [==============================] - 0s 26ms/step


In [320]:
# vocab들 중에 각 단어일 확률
np.argmax(y_pred)

3066

In [321]:
tokenizer_review.index_word[np.argmax(y_pred)]

'점임'

## 뉴스기사 데이터

#### 데이터 로드

In [186]:
# 뉴스 기사 데이터
!pip install -U --no-cache-dir gdown --pre
!gdown --no-cookies --id 1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS
From (redirected): https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS&confirm=t&uuid=0f73d920-a61d-4af4-be8f-366e5b17fbba
To: /content/train_original.json
100% 800M/800M [00:07<00:00, 101MB/s]


In [187]:
data = pd.read_json('train_original.json')

news_data = pd.DataFrame(data)

In [188]:
news_data

,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"
...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]"
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]"
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]"
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]"


In [189]:
news = news_data['article_original'][:100]
news

0     [지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...
1     [서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...
2     [지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...
3     [서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...
4     [미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...
                            ...                        
95    [5.18 현장의 비밀요원이 5.18주장을 검증했다., 지난 14일 방송된 JTBC...
96    [[공주] (재)백제세계유산센터는 지난 2017년 첫 번째 일러스트 공모전에 이어 ...
97    [지난 14일 KBS1 '추적60분'이 '환상의 재테크? 태양광 발전의 그늘'이 방...
98    [체력 소모가 많은 여름철을 맞아 '인삼'으로 건강 관리를 하는 사람들이 늘고 있다...
99    [경기도가 평화시대를 맞아 통일문제를 보다 쉽고 재미있게 다가갈 수 있도록 통일전문...
Name: article_original, Length: 100, dtype: object

#### 텍스트 전처리

In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm


okt = Okt()

news_text = []
for i in tqdm(news):
    for a in i:
        news_text.append(re.sub('[^가-힣ㄱ-ㅣa-zA-Z|]', ' ', string = a))
news_text

In [ ]:
text = [okt.morphs(i) for i in tqdm(news_text)]
text

In [ ]:
news_doc = sum(text, [])
news_doc

#### 토크나이저

In [257]:
# 토크나이저 생성
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(split=' ', char_level=False)
tokenizer.fit_on_texts(news_doc)

In [ ]:
tokenizer.word_index

In [259]:
news_vocab_size = len(tokenizer.word_index)+1
news_vocab_size

6558

In [ ]:
news_list = []
for i in text:
    a = ' '.join(i)
    news_list.append(a)
news_list

#### 데이터 셋 구축

In [261]:
# 훈련 데이터 만들기 1. 뒤에 단어 추측할 수 있도록 문장 분할한 리스트 생성

sequences_list = []

for doc in news_list:
    line = tokenizer.texts_to_sequences([doc])[0] # 단어 > 정수로 변환

    # line = doc.split(' ') # 한글로 보기 편하려면 이 코드 사용> 나중에 정수 변환 필요
    for i in range(1, len(line)):
        sequence = line[:i+1]
        sequences_list.append(sequence)

sequences_list

[[86, 1645],
 [86, 1645, 25],
 [86, 1645, 25, 2187],
 [86, 1645, 25, 2187, 10],
 [86, 1645, 25, 2187, 10, 2188],
 [86, 1645, 25, 2187, 10, 2188, 3231],
 [86, 1645, 25, 2187, 10, 2188, 3231, 2189],
 [86, 1645, 25, 2187, 10, 2188, 3231, 2189, 12],
 [86, 1645, 25, 2187, 10, 2188, 3231, 2189, 12, 207],
 [86, 1645, 25, 2187, 10, 2188, 3231, 2189, 12, 207, 9],
 [86, 1645, 25, 2187, 10, 2188, 3231, 2189, 12, 207, 9, 1098],
 [86, 1645, 25, 2187, 10, 2188, 3231, 2189, 12, 207, 9, 1098, 552],
 [86, 1645, 25, 2187, 10, 2188, 3231, 2189, 12, 207, 9, 1098, 552, 1099],
 [86, 1645, 25, 2187, 10, 2188, 3231, 2189, 12, 207, 9, 1098, 552, 1099, 1100],
 [86,
  1645,
  25,
  2187,
  10,
  2188,
  3231,
  2189,
  12,
  207,
  9,
  1098,
  552,
  1099,
  1100,
  1101],
 [86,
  1645,
  25,
  2187,
  10,
  2188,
  3231,
  2189,
  12,
  207,
  9,
  1098,
  552,
  1099,
  1100,
  1101,
  151],
 [86,
  1645,
  25,
  2187,
  10,
  2188,
  3231,
  2189,
  12,
  207,
  9,
  1098,
  552,
  1099,
  1100,
  1101,
  15

In [262]:
print(sequences_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [263]:
# 훈련 데이터 만들기 2. 문장 길이가 균일하지 않아서 padding

padded = pad_sequences(sequences_list, padding = 'pre')
padded

array([[   0,    0,    0, ...,    0,   86, 1645],
       [   0,    0,    0, ...,   86, 1645,   25],
       [   0,    0,    0, ..., 1645,   25, 2187],
       ...,
       [   0,    0,    0, ...,    9, 2867,   20],
       [   0,    0,    0, ..., 2867,   20,   38],
       [   0,    0,    0, ...,   20,   38,   11]], dtype=int32)

In [264]:
# 훈련 데이터 만들기 3. x 데이터와 y 데이터 분리

X_train = padded[:, :-1]
y_train = padded[:, -1]

In [265]:
print (X_train[0], '\n', y_train[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0 86] 
 1645


In [266]:
X_train.shape, y_train.shape

((34798, 155), (34798,))

### 모델

In [267]:
# modeling

input = tf.keras.Input(shape = (155,))

a = tf.keras.layers.Embedding(vocab_size, 15)(input)
a = tf.keras.layers.GRU(30,return_sequences = False)(a)

output = tf.keras.layers.Dense(vocab_size, activation = 'softmax')(a)

model_1 = tf.keras.Model(inputs = input, outputs = output)

In [268]:
model_1.compile(loss = 'sparse_categorical_crossentropy',
              optimizer ='adam',
              metrics = ['acc'])

In [310]:
model_1.fit(X_train[:100], y_train[:100], batch_size = 64, epochs = 100, validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 0s 180ms/step - loss: 4.1434 - acc: 0.0375 - val_loss: 5.7974 - val_acc: 0.0500
Epoch 2/100
2/2 [==============================] - 0s 204ms/step - loss: 4.1429 - acc: 0.0375 - val_loss: 5.7972 - val_acc: 0.0500
Epoch 3/100
2/2 [==============================] - 0s 184ms/step - loss: 4.1430 - acc: 0.0375 - val_loss: 5.7968 - val_acc: 0.0500
Epoch 4/100
2/2 [==============================] - 0s 139ms/step - loss: 4.1426 - acc: 0.0375 - val_loss: 5.7963 - val_acc: 0.0500
Epoch 5/100
2/2 [==============================] - 0s 157ms/step - loss: 4.1427 - acc: 0.0375 - val_loss: 5.7972 - val_acc: 0.0500
Epoch 6/100
2/2 [==============================] - 0s 179ms/step - loss: 4.1420 - acc: 0.0375 - val_loss: 5.8001 - val_acc: 0.0500
Epoch 7/100
2/2 [==============================] - 0s 175ms/step - loss: 4.1420 - acc: 0.0375 - val_loss: 5.8034 - val_acc: 0.0500
Epoch 8/100
2/2 [==============================] - 0s 142ms/step - loss: 4.1413 - a